In [ ]:

from isaacsim import SimulationApp

simulation_app = SimulationApp({"headless": True})


[Warning] [simulation_app.simulation_app] fast shutdown not supported with jupyter notebooks
Starting kit application with the following args:  ['/home/ronim/isaacsim/_build/linux-x86_64/release/exts/isaacsim.simulation_app/isaacsim/simulation_app/simulation_app.py', '/home/ronim/isaacsim/_build/linux-x86_64/release/apps/isaacsim.exp.base.python.kit', '--/app/tokens/exe-path=/home/ronim/isaacsim/_build/linux-x86_64/release/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True', '--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--/app/fastShutdown=False', '--/app/installSignalHandlers=0', '--ext-folder', '/home/ronim/isaacsim/_build/linux-x86_64/release/exts', '--ext-folder', '/home/ronim/isaacsim/_build/linux-x86_64/release/apps', '--/physics/cudaDe

[9.850s] Simulation App Startup Complete
2025-06-26T09:22:59Z [10,846ms] [Warning] [omni.isaac.core.utils.numpy.maths] omni.isaac.core.utils.numpy.maths has been deprecated in favor of isaacsim.core.utils.numpy.maths. Please update your code accordingly.
2025-06-26T09:22:59Z [10,846ms] [Warning] [omni.isaac.core.utils.numpy.rotations] omni.isaac.core.utils.numpy.rotations has been deprecated in favor of isaacsim.core.utils.numpy.rotations. Please update your code accordingly.
2025-06-26T09:22:59Z [10,847ms] [Warning] [omni.isaac.core.utils.numpy.tensor] omni.isaac.core.utils.numpy.tensor has been deprecated in favor of isaacsim.core.utils.numpy.tensor. Please update your code accordingly.
2025-06-26T09:22:59Z [10,847ms] [Warning] [omni.isaac.core.utils.numpy.transformations] omni.isaac.core.utils.numpy.transformations has been deprecated in favor of isaacsim.core.utils.numpy.transformations. Please update your code accordingly.
2025-06-26T09:23:02Z [14,208ms] [Warning] [isaacsim.sensor

In [6]:

import carb
import omni
from pxr import Usd, UsdGeom
import omni.usd
from isaacsim.core.api import World
from pxr import UsdLux, Gf
from isaacsim.core.api.objects import DynamicCuboid
import numpy as np
from isaacsim.core.prims import RigidPrim
from omni.isaac.sensor import Camera
import omni.isaac.core.utils.numpy.rotations as rot_utils


world = World(stage_units_in_meters=1.0)
world.scene.add_default_ground_plane()
world.reset()
stage = omni.usd.get_context().get_stage()

cube = world.scene.add(
    DynamicCuboid(
        prim_path="/cube",
        name="cube",
        position=np.array([0.0, 0, 0.0]),
        scale=np.array([0.6, 0.5, 0.2]),
        size=1.0,
        color=np.array([255, 0, 0]),
    )
)


camera = Camera(
    prim_path="/World/camera",
    position=np.array([0.0, 0.0, 50.0]),
    frequency=20,
    resolution=(256, 256),
    orientation=rot_utils.euler_angles_to_quats(np.array([0, 90, 0]), degrees=True),
)

camera.initialize()
rigid_cube = RigidPrim("/cube")
velocity = np.array([0.0, 0.0, 0.0])
pressed_keys = set()



def keyboard_event(event, *args, **kwargs):
    global pressed_keys
    if event.type == carb.input.KeyboardEventType.KEY_PRESS:
        pressed_keys.add(event.input)
    elif event.type == carb.input.KeyboardEventType.KEY_RELEASE:
        if event.input in pressed_keys:
            pressed_keys.remove(event.input)   



appwindow = omni.appwindow.get_default_app_window()
input_inter = carb.input.acquire_input_interface()
input_inter.subscribe_to_keyboard_events(appwindow.get_keyboard(), keyboard_event)
camera.add_motion_vectors_to_frame()


In [13]:
from fastapi import FastAPI
import matplotlib.pyplot as plt
from starlette.responses import StreamingResponse
import io
import numpy as np
from PIL import Image
import time

app = FastAPI()


def generate_frames():
    for idx in range(1,100,1):
        # Advance Isaac Sim simulation
        world.step(render=True)

        # Move object based on key input
        direction = np.array([0.0, 0.0, 0.0])
        if carb.input.KeyboardInput.A in pressed_keys:
            direction += np.array([0.0, 1.0, 0.0])
        if carb.input.KeyboardInput.D in pressed_keys:
            direction += np.array([0.0, -1.0, 0.0])
        if carb.input.KeyboardInput.W in pressed_keys:
            direction += np.array([1.0, 0.0, 0.0])
        if carb.input.KeyboardInput.S in pressed_keys:
            direction += np.array([-1.0, 0.0, 0.0])

        if np.linalg.norm(direction) > 0:
            velocity = direction / np.linalg.norm(direction) * 10
        else:
            velocity = np.array([0.0, 0.0, 0.0])

        rigid_cube.set_linear_velocity(velocity)

        # Capture image from Isaac Sim camera
        if idx > 10:
            frame = camera.get_rgba()[:, :, :3]
            image = Image.fromarray(frame, "RGB")

            buffer = io.BytesIO()
            image.save(buffer, format="JPEG")
            jpeg_bytes = buffer.getvalue()

            yield (
                b"--frame\r\n"
                b"Content-Type: image/jpeg\r\n\r\n" +
                jpeg_bytes + b"\r\n"
            )

        time.sleep(1/10)  # ~10 FPS


@app.get("/video")
def video_feed():
    return StreamingResponse(generate_frames(), media_type="multipart/x-mixed-replace; boundary=frame")


In [ ]:


from fastapi import FastAPI
from starlette.responses import StreamingResponse
import numpy as np
from PIL import Image
import io
import time

app = FastAPI()

def generate_frames():
    while True:
        # 1. Create a random image (numpy array)
        array = np.random.randint(0, 255, (240, 320, 3), dtype=np.uint8)
        image = Image.fromarray(array, "RGB")

        # 2. Encode as JPEG in memory
        buffer = io.BytesIO()
        image.save(buffer, format="JPEG")
        jpeg_bytes = buffer.getvalue()

        # 3. Yield the MJPEG chunk
        yield (
            b"--frame\r\n"
            b"Content-Type: image/jpeg\r\n\r\n" +
            jpeg_bytes + b"\r\n"
        )

        # Optional: slow down frame rate
        time.sleep(1 / 10)  # ~10 FPS

@app.get("/video")
def video_feed():
    return StreamingResponse(
        generate_frames(),
        media_type="multipart/x-mixed-replace; boundary=frame"
    )


In [ ]:
import matplotlib.pyplot as plt
from pxr import UsdLux, Sdf

# Assume your camera is at /World/MyCamera
stage = omni.usd.get_context().get_stage()
light_path = "/World/MyCamera/SpotLight"

# Create the light
light = UsdLux.SphereLight.Define(stage, Sdf.Path(light_path))
light.CreateIntensityAttr(5000.0)
light.CreateColorAttr((1.0, 1.0, 1.0))
light.CreateRadiusAttr(0.5)
plt.ion()
figure, ax = plt.subplots()

i = 0
camera.add_motion_vectors_to_frame()

for i in np.arange(1,51,1):
    world.step(render=True)
    # print(camera.get_current_frame())
    if i % 5 == 0:
        points_2d = camera.get_image_coords_from_world_points(
            np.array([cube_3.get_world_pose()[0], cube_2.get_world_pose()[0]])
        )
        points_3d = camera.get_world_points_from_image_coords(points_2d, np.array([24.94, 24.9]))
        # print(points_2d)
        # print(points_3d)
        # --- MODIFIED PLOTTING ---
        ax.clear() # Clear the previous image
        ax.imshow(camera.get_rgba()[:, :, :3])
        figure.canvas.draw()
        figure.canvas.flush_events()
        # -------------------------
        # print(camera.get_current_frame()["motion_vectors"])
        
    # if my_world.is_playing():
    #     if my_world.current_time_step_index == 0:
    #         my_world.reset()
    print(i)



simulation_app.close()